In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import statsmodels.tsa as tsa

from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV, LogisticRegression, LinearRegression
from sklearn.model_selection import TimeSeriesSplit, train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from eptest_main import read_ep_data, get_ct_dfs, adjust_ct_dfs, get_rolling_cts, test_adfuller, RegressionMain
from eptest_main import CT_SWAP_MAP, CT_SIZES, COT_COLS, PRICE_DUR_COLS

legacy, tff, futures, swaps = read_ep_data()


In [2]:
futures['FV'].head()

field          PX_LAST  FUT_EQV_DUR_NOTL  OPEN_INT
2010-01-01  115.492188            4.4430     11938
2010-01-04  114.546875            4.4403    748097
2010-01-05  114.945312            4.4415    759207
2010-01-06  114.906250            4.4414    776474
2010-01-07  114.796875            4.4410    783076

In [3]:
idx = pd.IndexSlice
futures.loc[:, idx[:, 'PX_LAST']].tail()

ct                  TU          FV          TY         US         WN
field          PX_LAST     PX_LAST     PX_LAST    PX_LAST    PX_LAST
2020-10-14  110.437500  125.890625  139.218750  175.37500  220.15625
2020-10-15  110.437500  125.843750  139.125000  175.25000  219.84375
2020-10-16  110.429688  125.820312  139.062500  174.90625  218.93750
2020-10-19  110.425781  125.757812  138.921875  174.50000  218.25000
2020-10-20  110.425781  125.718750  138.703125  173.50000  215.81250

In [4]:

# look at historical CTD mty?
# wish i could get invoice spreads data more easily

# break the data as of measurement for initial modeling purposes
ct_dfs = get_ct_dfs(legacy, tff, futures, swaps)

ct_dfs['US'].head()


Com    NonCom   NonRep        AM  LevFunds   Dealers  \
2010-01-01  115151.0 -103690.0 -11461.0  327844.0 -157049.0 -155350.0   
2010-01-04  115151.0 -103690.0 -11461.0  327844.0 -157049.0 -155350.0   
2010-01-05  123354.0 -101533.0 -21821.0  327142.0 -152979.0 -146571.0   
2010-01-06  123354.0 -101533.0 -21821.0  327142.0 -152979.0 -146571.0   
2010-01-07  123354.0 -101533.0 -21821.0  327142.0 -152979.0 -146571.0   

            OtherRep    PX_LAST  FUT_EQV_DUR_NOTL  OPEN_INT     15y     20y  
2010-01-01   -3984.0  115.37500           12.4921  670420.0  4.3458  4.4542  
2010-01-04   -3984.0  115.09375           12.4824  667206.0  4.3372  4.4431  
2010-01-05   -5771.0  116.03125           12.5146  658779.0  4.2645  4.3876  
2010-01-06   -5771.0  115.31250           12.4900  659893.0  4.3355  4.4631  
2010-01-07   -5771.0  115.28125           12.4889  651459.0  4.3520  4.4770

In [5]:
# function to compute dv01 weighted values and shift CoT data 3d forward after

adj_ct_dfs = adjust_ct_dfs(ct_dfs, swaps, oi_avg_len=125)

adj_ct_dfs['TY'].head()



Com    NonCom   NonRep       AM  LevFunds  Dealers  OtherRep  \
2010-01-01       NaN       NaN      NaN      NaN       NaN      NaN       NaN   
2010-01-04       NaN       NaN      NaN      NaN       NaN      NaN       NaN   
2010-01-05       NaN       NaN      NaN      NaN       NaN      NaN       NaN   
2010-01-06  171617.0 -136356.0 -35261.0  99582.0  -50214.0 -80946.0   66839.0   
2010-01-07  171617.0 -136356.0 -35261.0  99582.0  -50214.0 -80946.0   66839.0   

                Com_dv01  Com_pctAvgOI   NonCom_dv01  ...  NonRep_pctAvgOI  \
2010-01-01           NaN           NaN           NaN  ...              NaN   
2010-01-04           NaN           NaN           NaN  ...              NaN   
2010-01-05           NaN           NaN           NaN  ...              NaN   
2010-01-06  1.555238e+07           NaN -1.235694e+07  ...              NaN   
2010-01-07  1.556449e+07           NaN -1.236656e+07  ...              NaN   

                 AM_dv01  AM_pctAvgOI  LevFunds_dv01  LevFunds_pctAvgOI  \
2010-01-01           NaN          NaN            NaN                NaN   
2010-01-04           NaN          NaN            NaN                NaN   
2010-01-05           NaN          NaN            NaN                NaN   
2010-01-06  9.024382e+06          NaN  -4.550524e+06                NaN   
2010-01-07  9.031409e+06          NaN  -4.554068e+06                NaN   

            Dealers_dv01  Dealers_pctAvgOI  OtherRep_dv01  OtherRep_pctAvgOI  \
2010-01-01           NaN               NaN            NaN                NaN   
2010-01-04           NaN               NaN            NaN                NaN   
2010-01-05           NaN               NaN            NaN                NaN   
2010-01-06 -7.335539e+06               NaN   6.057125e+06                NaN   
2010-01-07 -7.341251e+06               NaN   6.061842e+06                NaN   

                7y  
2010-01-01  3.5285  
2010-01-04  3.4801  
2010-01-05  3.3975  
2010-01-06  3.4545  
2010-01-07  3.4765  

[5 rows x 22 columns]

In [10]:
# resample on Friday's with 'window' sized rolling weekly differences
r = get_rolling_cts(adj_ct_dfs, '_dv01', 4)

# determine stationarity of features
for k, df in r.items():
    print(k, test_adfuller(df, maxlag=1))
for k, df in r.items():
    print(k, df.corr())

FV                    tstat          pval
Com_dv01      -10.470696  1.286637e-18
NonCom_dv01   -10.416175  1.752089e-18
NonRep_dv01   -11.303989  1.286738e-20
AM_dv01        -9.750801  8.025815e-17
LevFunds_dv01 -10.675903  4.054705e-19
Dealers_dv01  -10.560516  7.750055e-19
OtherRep_dv01  -9.875223  3.898650e-17
5y             -8.991446  6.874370e-15
TU                    tstat          pval
Com_dv01      -11.861426  6.832339e-22
NonCom_dv01   -11.617248  2.432153e-21
NonRep_dv01   -13.320193  6.481769e-25
AM_dv01       -11.141769  3.095542e-20
LevFunds_dv01 -10.085788  1.156411e-17
Dealers_dv01  -11.126475  3.364340e-20
OtherRep_dv01 -10.610182  5.861342e-19
2y             -9.137676  2.905878e-15
TY                    tstat          pval
Com_dv01       -9.779250  6.802792e-17
NonCom_dv01   -10.506264  1.052350e-18
NonRep_dv01   -10.457455  1.386732e-18
AM_dv01       -10.348364  2.575255e-18
LevFunds_dv01 -11.086427  4.185721e-20
Dealers_dv01  -10.941538  9.269361e-20
OtherRep_dv01 -1

In [17]:
# rolling regressions
is_start_dt = dt.date(2012, 1, 1)

ct = 'FV'
target = '5y'
num_features = ['AM_dv01', 'LevFunds_dv01', 'Dealers_dv01']
data = r[ct].copy(deep=True).loc[is_start_dt:]
X = data[num_features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)
is_end_dt = y_test.index[-1]

##
HERE
WE NEED TO NORMALIZE THESE FEATURES FIRST BEORE USING REGULARIZATION
START WITH SKLEARN THEN WE CAN MOVE ONTO OUR OWN IMPLEMENTATION TO RETRIEVE PREDICTIONS
##
# pred, dates, betas = RegressionMain(r[ct], target, features, 13, 'Ridge', embargo_size=0,
#                                    max_train_size=26, logpx=False, ewm_span=None, verbose=True, alpha_override=None)
# betas_df = pd.DataFrame(betas, index=dates, columns=['const']+features)
# resid = r[ct].loc[:, target] - pred

# fig, ax = plt.subplots(nrows=2)
# betas_df.plot(ax=ax[0])
# resid.plot(ax=ax[1])
# plt.show()

2012-01-06   -0.0470
2012-01-13   -0.1100
2012-01-20   -0.1640
2012-01-27   -0.1830
2012-02-03   -0.1760
2012-02-10   -0.0100
2012-02-17   -0.0640
2012-02-24    0.1265
2012-03-02    0.0600
2012-03-09    0.0475
2012-03-16    0.2065
2012-03-23    0.1655
2012-03-30    0.1760
2012-04-06    0.0240
2012-04-13   -0.2030
2012-04-20   -0.1970
2012-04-27   -0.1535
2012-05-04   -0.0960
2012-05-11   -0.0485
2012-05-18   -0.0440
2012-05-25   -0.0210
2012-06-01   -0.0945
2012-06-08   -0.0775
2012-06-15   -0.1505
2012-06-22   -0.0905
2012-06-29   -0.0220
2012-07-06   -0.1155
2012-07-13   -0.0895
2012-07-20   -0.1990
2012-07-27   -0.1110
               ...  
2018-01-12    0.1947
2018-01-19    0.2331
2018-01-26    0.2991
2018-02-02    0.3544
2018-02-09    0.2463
2018-02-16    0.2010
2018-02-23    0.1831
2018-03-02    0.0426
2018-03-09    0.1472
2018-03-16    0.0677
2018-03-23    0.0251
2018-03-30   -0.0234
2018-04-06   -0.0611
2018-04-13    0.0108
2018-04-20    0.1670
2018-04-27    0.2047
2018-05-04   

# ToDo

## Data
### General
    - Check for general numerical issues
        - US June 2015 contract had a delivery gap of ~5yrs relative to March15 due to tsy issuance in early 2000s
        - see: https://www.cmegroup.com/trading/interest-rates/files/mar-15-jun-15-roll-analysis.pdf
        - the CTD MM swap then gets closer to the 20y... consider interpolating
    - Create offset series for Fridays, lets just shift it 3 days (data stamped tuesday but reported friday+ if holiday)
    - Split train+cv and full test data. Test data lets do 2010-2011 and Oct. 2018+
### Feature Generation:
Start evaluating training data visually, we need to work on stationary time series so let's do the transforms upfront:
        - DV01-weighted positions or % OI will be more informative, dur*px/100*ct_size/10000
        - Rolling x-week changes in transformed CoT data
            - Consider PCA of this CoT data and look at consistency of loadings over time
        - Rolling x-week changes in swap data
        - Indicator variables of whether the CoT change was 'same-way' as market move
    
    

## Modelling
### Feature Selection
Let's do feature selection second- random forest type feature importance algos 
    - First pass we can look at a few sets of x-lagged 1m changes(e.g. 1m lagged 1m changes, 2m lagged 1m changes, etc)
    
### Regression
Features need to be fit on a weekly basis- this data is weekly
    - We could consider either modeling on Tuesday data or Friday, but need to evaluate residuals based on Friday
    - Residual evaluation can be daily, not sure how valuable that'd be
Simple setups first
    - lagged aggregated changes, including conditional on same way stuff
More complicated
    - Maybe we do our own type of gridsearch CV with the parameter being the length of the window to aggregate changes

## Extra Credit
Can we use some classification algorithms to filter when to trade or not to trade

In [ ]:
ct = 'FV'
target = '5y'
num_features = ['AM_dv01', 'LevFunds_dv01', 'Dealers_dv01']
cat_features = []

# standardize variables, going to use full sample normalization don't think it'll matter much 
#    since the data is already fairly stationary
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scale', StandardScaler(with_mean=True))
])
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
#combine different feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])